# A1. Models and Primitives
http://nbviewer.jupyter.org/github/bokeh/bokeh-notebooks/blob/master/tutorial/A1%20-%20Models%20and%20Primitives.ipynb

In [1]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

output_notebook()

Loading BokehJS ...

## Overview
Although typically referred to by a single name, Bokeh comprises two separate libraries.

The first component is a JavaScript library, BokehJS, that runs in the browser. This library is responsible for all of the rendering and user interaction. Its input is a collection of declarative JSON objects that comprise a “scenegraph”. The objects in this scenegraph describe everything that BokehJS should handle: what plots and widgets are present and in what arrangement, what tools and renderers and axes the plots will have, etc. These JSON objects are converted into JavaScript objects in the browser.

The second component is a library in Python (or other languages) that can generate the JSON described above. In the Python Bokeh library, this is accomplished at the lowest level by exposing a set of Model subclasses that exactly mirror the set of BokehJS Models that are use in the browser. Most of the models are very simple, usually consisting of a few property attributes and no methods. Model attributes can either be configured when the model is created, or later by setting attribute values on the model object:

**properties can be configured when a model object is initialized**  
```
glyph = Rect(x="x", y="y2", w=10, h=20, line_color=None)
```
**properties can be configured by assigning values to attributes on exitsting models**  
```
glyph.fill_alpha = 0.5
glyph.fill_color = "navy"
```
These methods of configuration work in general for all Bokeh models.

Bokeh models are eventually collected into a Bokeh Document for serialization between BokehJS and the Bokeh Python bindings. As an example, the following image shows an example document for simple plot:

<img src='http://nbviewer.jupyter.org/github/bokeh/bokeh-notebooks/blob/master/tutorial/assets/document.svg'>


Such a document could be created at a high level using the bokeh.plotting API, which automatically assembles any models such as axes and grids and toolbars in a reasonable way. However it is also possible to assemble all the components "by hand" using the low-level bokeh.models API. Using the bokeh.models interface provides complete control over how Bokeh plots and Bokeh widgets are put together and configured. However, it provides no help with assembling the models in meaningful or correct ways. It is entirely up to developers to build the scenegraph “by hand”.

For more information about the details of all Bokeh models, consult the Reference Guide.

## Example Walkthrough

In [2]:
from bokeh.sampledata.sprint import sprint
sprint[:10]

,Name,Country,Medal,Time,Year
0,Usain Bolt,JAM,GOLD,9.63,2012
1,Yohan Blake,JAM,SILVER,9.75,2012
2,Justin Gatlin,USA,BRONZE,9.79,2012
3,Usain Bolt,JAM,GOLD,9.69,2008
4,Richard Thompson,TRI,SILVER,9.89,2008
5,Walter Dix,USA,BRONZE,9.91,2008
6,Justin Gatlin,USA,GOLD,9.85,2004
7,Francis Obikwelu,POR,SILVER,9.86,2004
8,Maurice Greene,USA,BRONZE,9.87,2004
9,Maurice Greene,USA,GOLD,9.87,2000


In [3]:
from bokeh.io import output_notebook, show
from bokeh.models.glyphs import Circle, Text
from bokeh.models import ColumnDataSource, Range1d, DataRange1d, Plot

### Setting up Data

In [4]:
abbrev_to_country = {
    "USA": "United States",
    "GBR": "Britain",
    "JAM": "Jamaica",
    "CAN": "Canada",
    "TRI": "Trinidad and Tobago",
    "AUS": "Australia",
    "GER": "Germany",
    "CUB": "Cuba",
    "NAM": "Namibia",
    "URS": "Soviet Union",
    "BAR": "Barbados",
    "BUL": "Bulgaria",
    "HUN": "Hungary",
    "NED": "Netherlands",
    "NZL": "New Zealand",
    "PAN": "Panama",
    "POR": "Portugal",
    "RSA": "South Africa",
    "EUA": "United Team of Germany",
}

fill_color = { "gold": "#efcf6d", "silver": "#cccccc", "bronze": "#c59e8a" }
line_color = { "gold": "#c8a850", "silver": "#b0b0b1", "bronze": "#98715d" }

def selected_name(name, medal, year):
    return name if medal == "gold" and year in [1988, 1968, 1936, 1896] else ""

t0 = sprint.Time[0]

sprint["Abbrev"]       = sprint.Country
sprint["Country"]      = sprint.Abbrev.map(lambda abbr: abbrev_to_country[abbr])
sprint["Medal"]        = sprint.Medal.map(lambda medal: medal.lower())
sprint["Speed"]        = 100.0/sprint.Time
sprint["MetersBack"]   = 100.0*(1.0 - t0/sprint.Time)
sprint["MedalFill"]    = sprint.Medal.map(lambda medal: fill_color[medal])
sprint["MedalLine"]    = sprint.Medal.map(lambda medal: line_color[medal])
sprint["SelectedName"] = sprint[["Name", "Medal", "Year"]].apply(tuple, axis=1).map(lambda args: selected_name(*args))

source = ColumnDataSource(sprint)

### Building in stages
Let's build up our plot in stages, stopping to check the output along the way to see how things look.

As we go through, note the three methods that Plot, Chart, and Figure all have:

- p.add_glyph
- p.add_tools
- p.add_layout

These are actually small convenience 

### Basic Plot with Just Glyphs
First we create just the Plot with a title and some basic styling applied, as well add a few Circle glyphs for the actual race data. To manually configure glyphs, we first create a glyph object (e.g., Text or Circle) that is configured with the visual properties we want as well as the data columns to use for coordinates, etc. Then we call plot.add_glyph with the glyph, and the data source that the glyph should use.

In [6]:
plot_options = dict(plot_width=800, plot_height=480, toolbar_location=None, outline_line_color=None)

medal_glyph = Circle(x="MetersBack", y="Year", size=10, fill_color="MedalFill", 
                     line_color="MedalLine", fill_alpha=0.5)

athlete_glyph = Text(x="MetersBack", y="Year", x_offset=10, text="SelectedName",
                     text_align="left", text_baseline="middle", text_font_size="9pt")

no_olympics_glyph = Text(x=7.5, y=1942, text=["No Olympics in 1940 or 1944"],
                         text_align="center", text_baseline="middle",
                         text_font_size="9pt", text_font_style="italic", text_color="silver")

xdr = Range1d(start=sprint.MetersBack.max()+2, end=0)  # +2 is for padding
ydr = DataRange1d(range_padding=0.05)  

plot = Plot(x_range=xdr, y_range=ydr, **plot_options)
plot.title.text = "Usain Bolt vs. 116 years of Olympic sprinters"
plot.add_glyph(source, medal_glyph)
plot.add_glyph(source, athlete_glyph)
plot.add_glyph(no_olympics_glyph)
show(plot)

### Adding Axes and Grids

In [7]:
from bokeh.models import Grid, LinearAxis, SingleIntervalTicker

xdr = Range1d(start=sprint.MetersBack.max()+2, end=0)  # +2 is for padding
ydr = DataRange1d(range_padding=0.05)  

plot = Plot(x_range=xdr, y_range=ydr, **plot_options)
plot.title.text = "Usain Bolt vs. 116 years of Olympic sprinters"
plot.add_glyph(source, medal_glyph)
plot.add_glyph(source, athlete_glyph)
plot.add_glyph(no_olympics_glyph)

xticker = SingleIntervalTicker(interval=5, num_minor_ticks=0)
xaxis = LinearAxis(ticker=xticker, axis_line_color=None, major_tick_line_color=None,
                   axis_label="Meters behind 2012 Bolt", axis_label_text_font_size="10pt", 
                   axis_label_text_font_style="bold")
plot.add_layout(xaxis, "below")

xgrid = Grid(dimension=0, ticker=xaxis.ticker, grid_line_dash="dashed")
plot.add_layout(xgrid)

yticker = SingleIntervalTicker(interval=12, num_minor_ticks=0)
yaxis = LinearAxis(ticker=yticker, major_tick_in=-5, major_tick_out=10)
plot.add_layout(yaxis, "right")

show(plot)

### Adding a Hover Tool

In [8]:
from bokeh.models import HoverTool

In [9]:
tooltips = """
<div>
    <span style="font-size: 15px;">@Name</span>&nbsp;
    <span style="font-size: 10px; color: #666;">(@Abbrev)</span>
</div>
<div>
    <span style="font-size: 17px; font-weight: bold;">@Time{0.00}</span>&nbsp;
    <span style="font-size: 10px; color: #666;">@Year</span>
</div>
<div style="font-size: 11px; color: #666;">@{MetersBack}{0.00} meters behind</div>
"""

In [10]:
xdr = Range1d(start=sprint.MetersBack.max()+2, end=0)  # +2 is for padding
ydr = DataRange1d(range_padding=0.05)  

plot = Plot(x_range=xdr, y_range=ydr, **plot_options)
plot.title.text = "Usain Bolt vs. 116 years of Olympic sprinters"
medal = plot.add_glyph(source, medal_glyph)  # we need this renderer to configure the hover tool
plot.add_glyph(source, athlete_glyph)
plot.add_glyph(no_olympics_glyph)

xticker = SingleIntervalTicker(interval=5, num_minor_ticks=0)
xaxis = LinearAxis(ticker=xticker, axis_line_color=None, major_tick_line_color=None,
                   axis_label="Meters behind 2012 Bolt", axis_label_text_font_size="10pt", 
                   axis_label_text_font_style="bold")
plot.add_layout(xaxis, "below")

xgrid = Grid(dimension=0, ticker=xaxis.ticker, grid_line_dash="dashed")
plot.add_layout(xgrid)

yticker = SingleIntervalTicker(interval=12, num_minor_ticks=0)
yaxis = LinearAxis(ticker=yticker, major_tick_in=-5, major_tick_out=10)
plot.add_layout(yaxis, "right")

In [11]:
hover = HoverTool(tooltips=tooltips, renderers=[medal])
plot.add_tools(hover)

In [12]:
show(plot)

## Custom User Models

### Implement the JavaScript Model

In [13]:
JS_CODE = """
import {div, empty} from "core/dom"

# The "core/properties" module has all the property types
import * as p from "core/properties"

# We will subclass in JavaScript from the same class that was subclassed
# from in Python
import {LayoutDOM, LayoutDOMView} from "models/layouts/layout_dom"

# This model will actually need to render things, so we must provide
# view. The LayoutDOM model has a view already, so we will start with that
export class CustomView extends LayoutDOMView

  initialize: (options) ->
    super(options)

    @render()

    # Set Backbone listener so that when the Bokeh slider has a change
    # event, we can process the new data
    @connect(@model.slider.change, () => @render())

  render: () ->
    # Backbone Views create <div> elements by default, accessible as @el.
    # Many Bokeh views ignore this default <div>, and instead do things
    # like draw to the HTML canvas. In this case though, we change the
    # contents of the <div>, based on the current slider value.
    empty(@el)
    @el.appendChild(div({
      style: {
        color: '#686d8e'
        'background-color': '#2a3153'
      }
    }, "#{@model.text}: #{@model.slider.value}"))

export class Custom extends LayoutDOM

  # If there is an associated view, this is boilerplate.
  default_view: CustomView

  # The ``type`` class attribute should generally match exactly the name
  # of the corresponding Python class.
  type: "Custom"

  # The @define block adds corresponding "properties" to the JS model. These
  # should basically line up 1-1 with the Python model class. Most property
  # types have counterparts, e.g. bokeh.core.properties.String will be
  # p.String in the JS implementation. Where the JS type system is not yet
  # as rich, you can use p.Any as a "wildcard" property type.
  @define {
    text:   [ p.String ]
    slider: [ p.Any    ]
  }
"""

### Define the Python Model

In [14]:
from bokeh.core.properties import String, Instance
from bokeh.models import LayoutDOM, Slider

class Custom(LayoutDOM):

    __implementation__ = JS_CODE

    text = String(default="Custom text")

    slider = Instance(Slider)

### Use the Python Model

In [15]:
from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import Slider

slider = Slider(start=0, end=10, step=0.1, value=0, title="value")

custom = Custom(text="Special Slider Display", slider=slider)

layout = column(slider, custom)

# Currently a bug preventing custom extensions working in notebooks
#show(layout)